In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

RESULTS_DIR = 'experiments_results'
CSV_FILENAME = 'test_results.csv'
CSV_PATH = os.path.join(RESULTS_DIR, CSV_FILENAME)


def plot_metric_comparison(df, metric='mAP50', title='Comparação de mAP@50 entre Modelos'):
    """Cria um gráfico de barras para comparar uma métrica específica."""
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(14, 8))

    df_sorted = df.sort_values(by=metric, ascending=False)

    sns.barplot(x='Model_Label', y=metric, hue='Type', data=df_sorted, ax=ax, dodge=False)

    ax.set_title(title, fontsize=16, weight='bold')
    ax.set_xlabel('Modelo', fontsize=12)
    ax.set_ylabel(metric, fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    output_path = os.path.join(RESULTS_DIR, f'{metric.lower()}_comparison.png')
    plt.savefig(output_path)
    print(f"Gráfico salvo em: {output_path}")
    plt.show()


def plot_accuracy_vs_speed(df, metric='mAP50', speed_metric='FPS'):
    """Cria um gráfico de dispersão para visualizar o trade-off entre acurácia e velocidade."""
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 8))

    sns.scatterplot(x=speed_metric, y=metric, hue='Type', style='Type', s=150, data=df, ax=ax)

    for i, row in df.iterrows():
        ax.text(row[speed_metric] * 1.01, row[metric], row['Model_Label'], fontsize=9)

    ax.set_title(f'Trade-off: {metric} vs. {speed_metric}', fontsize=16, weight='bold')
    ax.set_xlabel(f'Velocidade ({speed_metric})', fontsize=12)
    ax.set_ylabel(f'Acurácia ({metric})', fontsize=12)
    ax.legend(title='Tipo de Modelo')
    plt.tight_layout()

    output_path = os.path.join(RESULTS_DIR, 'accuracy_vs_speed.png')
    plt.savefig(output_path)
    print(f"Gráfico salvo em: {output_path}")
    plt.show()


def main():
    """Função principal para carregar os dados e gerar os gráficos."""
    if not os.path.exists(CSV_PATH):
        print(f"Erro: Arquivo '{CSV_PATH}' não encontrado.")
        return

    df = pd.read_csv(CSV_PATH)

    def create_label(row):
        if row['Type'] == 'Fine-Tuned':
            try:
                return os.path.basename(os.path.dirname(os.path.dirname(row['Path'])))
            except:
                return row['Model']
        return row['Model'].replace('.pt', '')

    df['Model_Label'] = df.apply(create_label, axis=1)

    df_filtered = df[df['mAP50'] > 0.1].copy()
    
    print("--- Dados Filtrados para Visualização ---")
    print(df_filtered[['Model_Label', 'Type', 'mAP50', 'FPS']].to_string())
    print("-" * 40)

    plot_metric_comparison(df_filtered, metric='mAP50', title='Comparação de mAP@50 (Fine-Tuned vs. Baseline)')
    plot_metric_comparison(df_filtered, metric='FPS', title='Comparação de FPS (Fine-Tuned vs. Baseline)')
    plot_accuracy_vs_speed(df_filtered)


if __name__ == '__main__':
    main()